In [ ]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

### LOAD AND CLEAN DATA

In [ ]:
# Cost of Living composite index by MSA (1990Q1-2022Q3)
coli_column_names = ['YEAR', 'QUARTER', 'CBSA_CODE', 'COMPOSITE_INDEX']
coli_raw = pd.read_excel('coli_raw/COLIHistorical.xlsx', sheet_name=0, usecols=coli_column_names)
coli_df = coli_raw.loc[coli_raw['QUARTER'] == 'Annual', ['YEAR', 'CBSA_CODE','COMPOSITE_INDEX']]
coli_df


# BEA Personal Income per Capita & Population
income_df = pd.read_csv('bea_raw/per_capita_income.csv', index_col='Unnamed: 0').rename(columns={'GeoFips':'FIPS_CODE', 'TimePeriod':'YEAR'})
income_df


# IPUMS measure of education
ipums_raw = pd.read_csv('ipums_raw/cps_00001.csv')
ipums_df = ipums_raw.loc[(ipums_raw['YEAR'] >= 2000), :]
ipums_df['COUNTY'] = ipums_df['COUNTY'].astype(int)
## construct within county variables
ipums_df = ipums_df.loc[:,['YEAR','COUNTY','EDUC','FTOTVAL','CAIDNW','OFFPOV']]
ipums_df = ipums_df.loc[ipums_df['EDUC'] != '999', :]
ipums_df['BACHELORS'] = ipums_df['EDUC'] >= 111
ipums_df['BACHELORS_SHARE'] = ipums_df.groupby(['YEAR','COUNTY'])['BACHELORS'].transform('mean')
ipums_df['POVERTY'] = ipums_df['OFFPOV'] == 1
ipums_df['POVERTY_SHARE'] = ipums_df.groupby(['YEAR','COUNTY'])['POVERTY'].transform('mean')
ipums_county_df = ipums_df.groupby(['YEAR','COUNTY'])[['BACHELORS_SHARE','POVERTY_SHARE']].last().reset_index()
ipums_county_df = ipums_df.rename(columns={'COUNTY':'FIPS_CODE'})
ipums_county_df


# Crosswalk to merge FIPS, ZIP, CBSA, etc.
crosswalk = pd.read_csv('crosswalks/cbsa2fipsxw.csv').loc[:,['cbsacode','csacode','cbsatitle','csatitle','fipsstatecode','fipscountycode']]
crosswalk['fips'] = crosswalk['fipsstatecode']*1000 + crosswalk['fipscountycode']
crosswalk = crosswalk.rename(columns={'cbsacode':'CBSA_CODE', 'fips':'FIPS_CODE'})
## crosswalk supplement for CBSAs that encompass multiple FIPS
crosswalk_supp = pd.read_csv('crosswalks/census_crosswalk.csv', skiprows=2).head(-4).rename(columns={'Metropolitan Division Code':'CBSA_CODE'})
crosswalk_supp['fips_supp'] = (crosswalk_supp['FIPS State Code']*1000 + crosswalk_supp['FIPS County Code']).astype(int)
crosswalk_supp['CBSA_CODE'] = pd.to_numeric(crosswalk_supp['CBSA_CODE'], errors='coerce').fillna(-99).astype(int)


# Joining all data
combined_df = coli_df.merge(crosswalk[['CBSA_CODE','FIPS_CODE']], on='CBSA_CODE', how='left')
combined_df = combined_df[['YEAR','CBSA_CODE','FIPS_CODE','COMPOSITE_INDEX']]
combined_df = combined_df.merge(crosswalk_supp[['CBSA_CODE','fips_supp']], on='CBSA_CODE', how='left')
combined_df['fips_supp'] = pd.to_numeric(combined_df['fips_supp'], errors='coerce').fillna(-99).astype(int)
combined_df['FIPS_CODE'] = combined_df['FIPS_CODE'].fillna(combined_df['fips_supp'])
combined_df = combined_df.drop(columns=['fips_supp']) # Columns collected: YEAR	CBSA_CODE	FIPS_CODE	COMPOSITE_INDEX

combined_df = combined_df.merge(income_df, on=['YEAR','FIPS_CODE'], how='left')
combined_df['FIPS_CODE'] = combined_df['FIPS_CODE'].astype(int) # Columns collected: YEAR	CBSA_CODE	FIPS_CODE	COMPOSITE_INDEX	INCOME_CAPITA POPULATION

combined_df = combined_df.merge(ipums_county_df[['YEAR','FIPS_CODE','BACHELORS_SHARE','POVERTY_SHARE']], on=['YEAR','FIPS_CODE'], how='left')
combined_df = combined_df.drop_duplicates().dropna() # ~1% of observations don't have BACHELORS_SHARE # Columns collected: YEAR	CBSA_CODE	FIPS_CODE	COMPOSITE_INDEX	INCOME_CAPITA POPULATION	BACHELORS_SHARE
# Saving combined data
combined_df.to_csv('combined.csv')


### ANALYSIS

In [ ]:
df = pd.read_csv('combined.csv')
df = df.sort_values(['CBSA_CODE','YEAR'])

# decimal to percent
df['BACHELORS_SHARE'] = df['BACHELORS_SHARE']*100
df['POVERTY_SHARE'] = df['POVERTY_SHARE']*100
# individuals to millions of individuals
df['POPULATION'] = df['POPULATION']/1_000_000
# log versions of columns
for col in ['COMPOSITE_INDEX','INCOME_CAPITA','POPULATION','BACHELORS_SHARE','POVERTY_SHARE']:
    df[f'{col}_log'] = np.log(df[col])


# Table 1: Summary Stats
round(df[['COMPOSITE_INDEX','INCOME_CAPITA','POPULATION','BACHELORS_SHARE','POVERTY_SHARE']].describe().drop(['25%', '50%', '75%']).T,2).to_csv('summary_stats.csv')

# Model 1 log-log
y = df['COMPOSITE_INDEX_log']
X = sm.add_constant(df['INCOME_CAPITA_log'])
model = sm.OLS(y, X).fit(cov_type='HC3')
model.summary()


# Model 1 log-log robustness
y = df['COMPOSITE_INDEX_log']
X = sm.add_constant(df[['INCOME_CAPITA_log','POPULATION','BACHELORS_SHARE','POVERTY_SHARE']])
model = sm.OLS(y, X).fit(cov_type='HC3')
model.summary()


# Model 2 County Fixed Effects using "Within Estimator"
time_avg = df.groupby('YEAR').mean()
# Merge the time averages back to the original DataFrame
within_df = pd.merge(df, time_avg, left_on='YEAR', right_index=True, suffixes=('', '_avg'))
# Subtract time averages from each variable
within_df['COMPOSITE_INDEX_log_demeaned'] = within_df['COMPOSITE_INDEX_log'] - within_df['COMPOSITE_INDEX_log_avg']
within_df['INCOME_CAPITA_log_demeaned'] = within_df['INCOME_CAPITA_log'] - within_df['INCOME_CAPITA_log_avg']
within_df['POPULATION_demeaned'] = within_df['POPULATION'] - within_df['POPULATION_avg']
within_df['BACHELORS_SHARE_demeaned'] = within_df['BACHELORS_SHARE'] - within_df['BACHELORS_SHARE_avg']
within_df['POVERTY_SHARE_demeaned'] = within_df['POVERTY_SHARE'] - within_df['POVERTY_SHARE_avg']
within_df = within_df[['CBSA_CODE','YEAR']+[col for col in within_df.columns if 'demeaned' in col]]

X = within_df[['INCOME_CAPITA_log_demeaned', 'POPULATION_demeaned','BACHELORS_SHARE_demeaned','POVERTY_SHARE_demeaned']] #
Y = within_df['COMPOSITE_INDEX_log_demeaned']
model = sm.OLS(Y, X).fit(cov_type='HC3')
model.summary()


# Model 2b County Fixed Effects using "Within Estimator" and Year Fixed Effects 
year_dummies = pd.get_dummies(within_df['YEAR'], prefix='year', drop_first=True)
within_dummy_df = pd.concat([within_df[['CBSA_CODE','YEAR','COMPOSITE_INDEX_log_demeaned','INCOME_CAPITA_log_demeaned', 'POPULATION_demeaned','BACHELORS_SHARE_demeaned','POVERTY_SHARE_demeaned']], year_dummies], axis=1)

X = within_dummy_df[['INCOME_CAPITA_log_demeaned', 'POPULATION_demeaned','BACHELORS_SHARE_demeaned','POVERTY_SHARE_demeaned', 'year_2019', 'year_2020', 'year_2021', 'year_2022']] #
Y = within_dummy_df['COMPOSITE_INDEX_log_demeaned']
model = sm.OLS(Y, X).fit(cov_type='HC3')
model.summary()


# Model 3 Year Fixed Effects Only using dummy variables
year_dummies = pd.get_dummies(df['YEAR'], prefix='year', drop_first=True)
dummy_df = pd.concat([df[['CBSA_CODE','YEAR','COMPOSITE_INDEX_log','INCOME_CAPITA_log','POPULATION']], year_dummies], axis=1)

X = sm.add_constant(dummy_df[['INCOME_CAPITA_log', 'POPULATION', 'year_2019', 'year_2020', 'year_2021', 'year_2022']]) #
Y = dummy_df['COMPOSITE_INDEX_log']
model = sm.OLS(Y, X).fit(cov_type='HC3')
model.summary()